In [4]:
import yfinance as yf
from curve import *
from balancer import *
from gmx import *

import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# To get new data, delete the existing file in 'eth' folder
def get_eth_price():
    eth_folder = 'eth'
    eth_file = os.path.join(eth_folder, 'eth_price_data.csv')

    # Check if the ETH price data file exists
    if not os.path.exists(eth_file):
        # If the file doesn't exist, download the ETH price data
        print("ETH price data not found. Downloading from yfinance...")
        end_date = (datetime.now() - timedelta(1)).strftime('%Y-%m-%d')
        eth_data = yf.download('ETH-USD', start='2020-01-01', end=end_date)
        eth_data.reset_index(inplace=True)
        eth_data = eth_data.rename(columns={'Date': 'timestamp', 'Adj Close': 'eth_price'})
        eth_data = eth_data[['timestamp', 'eth_price']]
        eth_data['timestamp'] = eth_data['timestamp'].dt.strftime('%Y-%m-%d')

        # Save the data to the eth folder
        if not os.path.exists(eth_folder):
            os.makedirs(eth_folder)
        eth_data.to_csv(eth_file, index=False)
    else:
        # If the file exists, load the data from the file
        eth_data = pd.read_csv(eth_file)

    return eth_data

crv_df = merge_crv_data()
gmx_df = merge_gmx_data()
bal_df = merge_bal_data()

eth_df = get_eth_price()

# Add token_name column to each DataFrame
crv_df['token_name'] = 'CRV'
gmx_df['token_name'] = 'GMX'
bal_df['token_name'] = 'BAL'

# Concatenate the DataFrames
combined_df = pd.concat([crv_df, gmx_df, bal_df])

# Merge with ETH price data
combined_df = pd.merge(combined_df, eth_df, on='timestamp', how='left')

# Print the combined DataFrame info to verify
print(combined_df.info())
print(combined_df.head())